In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models import BedrockModel

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))

In [ ]:
! uv run strands_claude.py '{"prompt": "What is the weather now?"}'

Preparing your agent for deployment on AgentCore Runtime

In [ ]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

What happens behind the scenes?
When you use BedrockAgentCoreApp, it automatically:

Creates an HTTP server that listens on the port 8080
Implements the required /invocations endpoint for processing the agent's requirements
Implements the /ping endpoint for health checks (very important for asynchronous agents)
Handles proper content types and response formats
Manages error handling according to the AWS standards

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_claude"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="pyproject.toml",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/strands_claude.py, bedrock_agentcore_name=strands_claude
Configuring BedrockAgentCore agent: strands_claude
Configuring BedrockAgentCore agent: strands_claude


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/Dockerfile
Generated .dockerignore: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/.dockerignore
Keeping 'strands_claude' as default agent
Generated .dockerignore: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/.dockerignore
Keeping 'strands_claude' as default agent
Bedrock AgentCore configured: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/.bedrock_agentcore.yaml
Bedrock AgentCore configured: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/bedrock-model/.dockerignore'), runtime='Docker', region='ap-southeast-2', account_id='381491838394', execution_role=None, ecr_repository=None, auto_create_ecr=True)

In [18]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude' to account 381491838394 (ap-southeast-2)
Starting CodeBuild ARM64 deployment for agent 'strands_claude' to account 381491838394 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_claude


✅ ECR repository available: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-strands_claude
Getting or creating execution role for agent: strands_claude
Using AWS region: ap-southeast-2, account ID: 381491838394
Role name: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-46ebe432b7
Getting or creating execution role for agent: strands_claude
Using AWS region: ap-southeast-2, account ID: 381491838394
Role name: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-46ebe432b7
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-46ebe432b7
Starting execution role creation process for agent: strands_claude
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-46ebe432b7
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-46ebe432b7
Starting execution role creation process for agent: strands_claude
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-46ebe432b7
Creatin

In [19]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude


'READY'

In [20]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

{'ResponseMetadata': {'RequestId': 'ed8be18e-ee92-41d3-ba9e-bf18195d7ccc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 17 Nov 2025 23:13:47 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ed8be18e-ee92-41d3-ba9e-bf18195d7ccc',
   'baggage': 'Self=1-691bac1f-040983063c9267ba1f8ae235,session.id=2f6b3af1-96f1-4baa-947b-7e369b36653d',
   'x-amzn-bedrock-agentcore-runtime-session-id': '2f6b3af1-96f1-4baa-947b-7e369b36653d',
   'x-amzn-trace-id': 'Root=1-691bac1f-7e8ef9f74263d4f979e6c34b;Parent=1a82fa169958666a;Sampled=1;Self=1-691bac1f-040983063c9267ba1f8ae235'},
  'RetryAttempts': 0},
 'runtimeSessionId': '2f6b3af1-96f1-4baa-947b-7e369b36653d',
 'traceId': 'Root=1-691bac1f-7e8ef9f74263d4f979e6c34b;Parent=1a82fa169958666a;Sampled=1;Self=1-691bac1f-040983063c9267ba1f8ae235',
 'baggage': 'Self=1-691bac1f-040983063c9267ba1f8ae235,session.id=2f6b3af1-96f1-4baa-947b-7e369b36653d',
 'contentType': 

In [21]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

The weather is currently sunny!

Invoking AgentCore Runtime with boto3

In [22]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The answer is 4.

Cleanup (Optional)

In [23]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-strands_claude',
 'strands_claude-Pm6cPM9dCj',
 'bedrock-agentcore-strands_claude')

In [24]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)